<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500 height=450/></p>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Продвинутый поток. Весна 2021</b></h3>

Это домашнее задание будет посвящено полноценному решению задачи машинного обучения.

Есть две части этого домашнего задания: 
* Сделать полноценный отчет о вашей работе: как вы обработали данные, какие модели попробовали и какие результаты получились (максимум 10 баллов). За каждую выполненную часть будет начислено определенное количество баллов.
* Лучшее решение отправить в соревнование на [kaggle](https://www.kaggle.com/c/advanced-dls-spring-2021/) (максимум 5 баллов). За прохождение определенного порогов будут начисляться баллы.


**Обе части будут проверяться в формате peer-review. Т.е. вашу посылку на степик будут проверять несколько других студентов и аггрегация их оценок будет выставлена. В то же время вам тоже нужно будет проверить несколько других учеников.**

**Пожалуйста, делайте свою работу чистой и понятной, чтобы облегчить проверку. Если у вас будут проблемы с решением или хочется совета, то пишите в наш чат в телеграме или в лс @runfme. Если вы захотите проаппелировать оценку, то пипшите в лс @runfme.**

**Во всех пунктах указания это минимальный набор вещей, которые стоит сделать. Если вы можете сделать какой-то шаг лучше или добавить что-то свое - дерзайте!**

# Как проверять?

Ставьте полный балл, если выполнены все рекомендации или сделано что-то более интересное и сложное. За каждый отсустствующий пункт из рекомендация снижайте 1 балл.

# **Метрика**

Перед решением любой задачи важно понимать, как будет оцениваться ваше решение. В данном случае мы используем стандартную для задачи классификации метрику ROC-AUC. Ее можно вычислить используя только предсказанные вероятности и истинные классы без конкретного порога классификации + она раотает даже если классы в данных сильно несбалансированны (примеров одного класса в десятки раз больше примеров длугого). Именно поэтому она очень удобна для соревнований.

Посчитать ее легко:


In [47]:
from sklearn.metrics import roc_auc_score

y_true = [
    0,
    1,
    1,
    0,
    1
]

y_predictions = [
    0.1,
    0.9,
    0.4,
    0.6,
    0.61
]

roc_auc_score(y_true, y_predictions)

# **Первая часть. Исследование**

In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
submission_path = '../input/advanced-dls-spring-2021/submission.csv'
test_path = '../input/advanced-dls-spring-2021/test.csv'
train_path = '../input/advanced-dls-spring-2021/train.csv'

## **Загрузка данных (2 балла)**

1) Посмотрите на случайные строчки. 

2) Посмотрите, есть ли в датасете незаполненные значения (nan'ы) с помощью data.isna() или data.info() и, если нужно, замените их на что-то. Будет хорошо, если вы построите табличку с количеством nan в каждой колонке.

In [49]:
data_train = pd.read_csv(train_path)

In [50]:
# Для вашего удобства списки с именами разных колонок

# Числовые признаки
num_features = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_features = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_features + cat_features
target_col = 'Churn'

Посмотрим на пять случайных строчек в датафрейме:

In [51]:
data_train.sample(5)

Определим количество NAN-ов в каждой колонке:

In [52]:
data_train.isna().sum()

Как видим, значений NAN в датафрейме не имеется. Однако имеются значения равные пробелу, которые не помечены как NAN, но являются пропущенными.

In [53]:
(data_train == ' ').sum()

Как видим, в TotalSpent 9 значений равно пробелу. Заменим их на np.nan и приведём к типу float:

In [54]:
data_train.TotalSpent = (np.where(data_train.TotalSpent == ' ', '1', data_train.TotalSpent)).astype(float)

# **Анализ данных (3 балла)**

1) Для численных призанков постройте гистограмму (*plt.hist(...)*) или boxplot (*plt.boxplot(...)*). Для категориальных посчитайте количество каждого значения для каждого признака. Для каждой колонки надо сделать *data.value_counts()* и построить bar диаграммы *plt.bar(...)* или круговые диаграммы *plt.pie(...)* (хорошо, елси вы сможете это сделать на одном гарфике с помощью *plt.subplots(...)*). 

2) Посмотрите на распределение целевой переменной и скажите, являются ли классы несбалансированными.

3) (Если будет желание) Поиграйте с разными библиотеками для визуализации - *sns*, *pandas_visual_analysis*, etc.

Второй пункт очень важен, потому что существуют задачи классификации с несбалансированными классами. Например, это может значить, что в датасете намного больше примеров 0 класса. В таких случаях нужно 1) не использовать accuracy как метрику 2) использовать методы борьбы с imbalanced dataset (обычно если датасет сильно несбалансирован, т.е. класса 1 в 20 раз меньше класса 0).

In [55]:
def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)

In [56]:
colors = get_cmap(3)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
fig.suptitle("Numerical features distribution", fontsize=(16))
for index, axis in enumerate(axes):
    sns.histplot(data=data_train[num_features[index]], ax=axis, color=colors(index))

In [57]:
colors = get_cmap(26)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 15))
fig.suptitle("Categorial features distribution", fontsize=(16))
for row_ind, axis_row in enumerate(axes):
  for ax_ind, axis in enumerate(axis_row):
    index = ax_ind + 4 * row_ind
    counts = data_train[cat_features[index]].value_counts()
    names = [name.split()[0] if index == 15 else name for name in counts.index]
    barplot = sns.barplot(x=names, y=counts.values, ax=axis, color=colors(index))
    barplot.set_xlabel(cat_features[index])
    barplot.set_ylabel('count')

Посмотрим на распределение целевой переменной:

In [58]:
data_train[target_col].hist()
pass

Данные классы не являются несбалансированными, хоть и виден перекос для 0 класса.

# **Применение линейных моделей (3 балла)**

1) Обработайте данные для того, чтобы к ним можно было применить LogisticRegression. Т.е. отнормируйте числовые признаки, а категориальные закодируйте с помощью one-hot-encoding'а. 

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC. 

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C). 


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

In [59]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

Создаём пайплайн из трансформера колонок и самой логистической регрессии:

In [60]:
X = data_train.iloc[:, :-1]
y = data_train.iloc[:, -1]

logreg_transformer = ColumnTransformer(transformers=[
                                ('num', RobustScaler(), num_features),
                                ('cat_label', OrdinalEncoder(), cat_features),
                                ('cat_OH', OneHotEncoder(), cat_features)
])

### Поиск оптимальных параметров
#pipeline = make_pipeline(logreg_transformer, LogisticRegression(n_jobs=-1))
#params = {
#    'logisticregression__max_iter' : [100, 300, 500],
#    'logisticregression__C' : [100, 10, 1, 0.1, 0.01, 0.001],
#    'logisticregression__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
#}

#grid = GridSearchCV(estimator=pipeline, param_grid=params, scoring='roc_auc', cv=3)
#grid.fit(X, y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2)
pipeline = make_pipeline(
                         logreg_transformer, 
                         LogisticRegression(**{'C' : 100, 'max_iter' : 500, 'solver' : 'saga'})
)
pipeline.fit(X_train, y_train)
probs = pipeline.predict_proba(X_valid)[:, 1]

In [61]:
print(f'Best ROC-AUC score - {roc_auc_score(y_valid, probs)}')

Выпишите какое лучшее качество и с какими параметрами вам удалось получить: лучшее качество - 0.8512, параметры - {C - 100, max_iter = 500, solver = saga} .

# **Применение градиентного бустинга (2 балла)**

Если вы хотите получить баллы за точный ответ, то стоит попробовать градиентный бустинг. Часто градиентный бустинг с дефолтными параметрами даст вам 80% результата за 0% усилий.

Мы будем использовать catboost, поэтому нам не надо кодировать категориальные признаки. catboost сделает это сам (в .fit() надо передать cat_features=cat_cols). А численные признаки нормировать для моделей, основанных на деревьях не нужно.

1) Разделите выборку на train/valid. Протестируйте catboost cо стандартными параметрами.

2) Протестируйте разные занчения параметроа количества деревьев и learning_rate'а и выберите лучшую по метрике ROC-AUC комбинацию. 

(Дополнительно) Есть некоторые сложности с тем, чтобы использовать CatBoostClassifier вместе с GridSearchCV, поэтому мы не просим использовать кроссвалидацию. Но можете попробовать)

In [62]:
!pip install catboost

In [63]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

X_test = pd.read_csv(test_path)
submission = pd.read_csv(submission_path)

### **1. Дефолтный CatBoostClassifier**

In [64]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [65]:
cat_default = CatBoostClassifier(cat_features=cat_features)
cat_default.fit(X_train, y_train, verbose=False)
y_predict = cat_default.predict_proba(X_valid)[:, 1]
cb_fpr, cb_tpr, _ = roc_curve(y_valid, y_predict)
# plot the roc curve for the model
plt.plot(cb_fpr, cb_tpr, linestyle='-', label='CatBoostClassifier')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
plt.show()

In [66]:
f"ROC-AUC score = {roc_auc_score(y_valid, y_predict)}"

### **2. CatBoostClassifier с настроенными параметрами**

In [67]:
### Большинство параметров взяты из другого ноутбука пользователя Sergey Troschiev
### Менял ТОЛЬКО количество итераций, оптимальным значением оказалось 284

params = {
    'n_estimators': 284,
    'max_depth': 4,
    'subsample': 0.6983901315995189,
    'l2_leaf_reg': 3.180242242411711,
    'random_strength': 1.2423130425640145,
    'eta': 0.04356020658096416,
    'min_data_in_leaf': 1,
    'grow_policy': 'Lossguide',
    'silent': True,
    'eval_metric' : 'AUC:hints=skip_train~false'
}

cat_classifier = CatBoostClassifier(cat_features=cat_features, **params)
cat_classifier.fit(X, y, verbose=False)
best_model = cat_classifier

Выпишите какое лучшее качество и с какими параметрами вам удалось получить

In [68]:
params = best_model.get_params()
params.pop('cat_features')
f"Лучшие параметры - {params}\nЛучше качество на кагле - {0.85469}"

### **Предсказания** ###

In [69]:
X_test.TotalSpent = np.where(X_test.TotalSpent == ' ', '1', X_test.TotalSpent).astype(float) # меняю на 1 по совету Grigory Soldatov
X_test.TotalSpent.fillna(X_test.TotalSpent.mean(), inplace=True)
submission['Churn'] = best_model.predict_proba(X_test)[:, 1]
submission.to_csv('./my_submission.csv', index=False)
submission.head(5)

ZhenDDOS, 0.85469


### **Kaggle (5 баллов)**

Как выставить баллы:

1) 1 >= roc auc > 0.84 это 5 баллов

2) 0.84 >= roc auc > 0.7 это 3 балла

3) 0.7 >= roc auc > 0.6 это 1 балл

4) 0.6 >= roc auc это 0 баллов

Для выполнения задания необходимо выполнить следующие шаги.

    Зарегистрироваться на платформе kaggle.com. Процесс выставления оценок будет проходить при подведении итогового рейтинга. Пожалуйста, укажите во вкладке Team -> Team name свои имя и фамилию в формате Имя_Фамилия (важно, чтобы имя и фамилия совпадали с данными на Stepik).
    Обучить модель, получить файл с ответами в формате .csv и сдать его в конкурс. Пробуйте и экспериментируйте. Обратите внимание, что вы можете выполнять до 20 попыток сдачи на kaggle в день.
    После окончания соревнования отправить в итоговый ноутбук с решением на степик.
    После дедлайна проверьте посылки других участников по критериям. Для этого надо зайти на степик, скачать их ноутбук и проверить скор в соревновании.

